### Import de dependências

In [9]:
import moviepy.editor as mpe
import numpy as np

### Abrir o video

In [10]:
video = mpe.VideoFileClip("projeto2C209.mp4")

### Inverter a imagem, no eixo, a cada 20s

In [11]:
def inverte_imagem(clip):
    def inverter_no_eixo_20_em_20_s(get_frame, t):
        #Cria intervalos de 20 segundos
        quociente = int(t) // 20
        if quociente % 2 == 1:
            frame = get_frame(t)
            #Inverte a imagem no eixo, caso esteja nos 20 segundos
            return np.flip(frame, axis=1)
        else:
            return get_frame(t)
    
    return clip.fl(inverter_no_eixo_20_em_20_s)

video_invertido_20 = inverte_imagem(video)

### Diminiur o som, de forma gradativa, a cada 30 segundos, de forma que os últimos 10 segundos fique em silêncio (sem som)

In [14]:
# Calcula e retorna o volume no tempo dado
def busca_volume(t, audio_shape):
    #Máscara que deixa o aúdio sem som nos últimos 10 segundos
    mask = t >= duracao_total - 10
    #Calcula intervalos de 30 segundos
    intervals = t // 30
    #Fator para diminuir gradativamente o som (diminui 20% do atual)
    factors = 1 - 0.2 * intervals
    #Aplica a máscara  que deixa sem som nos últimos 10 segundos
    factors[mask] = 0.0
    return np.tile(np.maximum(factors, 0.0), (audio_shape[1], 1)).T

# Aplica o volume a cada quadro de áudio
def aplicar_volume(get_audio_frame, t):
    audio_frame = get_audio_frame(t)
    fator = busca_volume(t, audio_frame.shape)
    return audio_frame * fator


duracao_total = video_invertido_20.duration
#Realiza a redução de volume
audio_reduzido = video_invertido_20.audio.fl(aplicar_volume) 
#Seta o aúdio no vídeo
video_audio_diminuindo = video_invertido_20.set_audio(audio_reduzido) 


### Após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80
### Por fim, insere os 20 segundos cortados no fim do video

In [15]:
video_0s_60s = video_audio_diminuindo.subclip(0,60)
video_60s_80s = video_audio_diminuindo.subclip(60,80)
video_80s_max = video_audio_diminuindo.subclip(80)

video_final = mpe.concatenate_videoclips([video_0s_60s,video_80s_max,video_60s_80s])

video_final.write_videofile('video_final.mp4')

Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
